In [1]:
from __future__ import print_function

import argparse
from collections import defaultdict
import logging


import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import sys
import yaml

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import h5py
import logging
import sys
from tqdm import tqdm

from ops_pytorch import (minibatch_discriminator, minibatch_output_shape, Dense3D,
                     calculate_energy, scale, InpaintingAttention)

from architectures_torch import build_Generator, build_Discriminator

import time

from torchsummary import summary

## Settaggio Parametri

In [2]:
nb_epochs = 50
batch_size = 256
latent_size = 100
verbose = True
no_attn = True

disc_lr =2e-5
gen_lr = 2e-4
adam_beta_1 = 0.5

yaml_file = "particles.yaml"

    # read in data file spec from YAML file
with open(yaml_file, 'r') as stream:
    try:
        s = yaml.safe_load(stream)
    except yaml.YAMLError as exc:

        raise exc
nb_classes = len(s.keys())
print(nb_classes)
a=[[p,f] for p,f in s.items()]
print(a)

1
[['positron', '../data/eplus.hdf5']]


## Caricamento dati 

In [3]:
def bit_flip(x, prob=0.05):
    """ flips a int array's values with some probability """
    x = np.array(x)
    selection = np.random.uniform(0, 1, x.shape) < prob
    x[selection] = 1 * np.logical_not(x[selection])
    return x

def _load_data(particle, datafile):


        d = h5py.File(datafile, 'r')

        # make our calo images channels-last
        first = np.expand_dims(d['layer_0'][:], -1)
        second = np.expand_dims(d['layer_1'][:], -1)
        third = np.expand_dims(d['layer_2'][:], -1)
        # convert to MeV
        energy = d['energy'][:].reshape(-1, 1) * 1000

        sizes = [
            first.shape[1], first.shape[2],
            second.shape[1], second.shape[2],
            third.shape[1], third.shape[2]
        ]

        y = [particle] * first.shape[0]

        d.close()

        return first, second, third, y, energy, sizes

In [4]:
first, second, third, y, energy, sizes = [
    np.concatenate(t) for t in [
        a for a in zip(*[_load_data(p, f) for p, f in s.items()])
    ]
]

sizes = sizes[:6].tolist()

# scale the energy depositions by 1000 to convert MeV => GeV
first, second, third, energy = [
    (X.astype(np.float32) / 1000)
    for X in [first, second, third, energy]
    ]
le = LabelEncoder()
y = le.fit_transform(y)

first = torch.tensor(first, dtype=torch.float32)[:10000].permute(0, 3, 1, 2)
second = torch.tensor(second, dtype=torch.float32)[:10000].permute(0, 3, 1, 2)
third = torch.tensor(third, dtype=torch.float32)[:10000].permute(0, 3, 1, 2)
y = torch.tensor(y, dtype=torch.long)[:10000]
energy = torch.tensor(energy, dtype=torch.float32, requires_grad=True)[:10000]
# Dataset and DataLoader
dataset = TensorDataset(first, second, third, energy,y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
x=next(iter(dataloader))
print(x[0].shape)
print(x[1].shape)
print(x[2].shape)
print(x[3].shape)
print(x[4].shape)
print(x[4])

torch.Size([256, 1, 3, 96])
torch.Size([256, 1, 12, 12])
torch.Size([256, 1, 12, 6])
torch.Size([256, 1])
torch.Size([256])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


## Discriminatore

In [13]:
class Discriminator(nn.Module):
    def __init__(self, sizes, nb_classes=1):
        super(Discriminator, self).__init__()
        self.nb_classes = nb_classes
        self.mbd = True
        self.sparsity = True
        self.sparsity_mbd = True

        self.layers = nn.ModuleList([
            build_Discriminator(mbd=self.mbd, sparsity=self.sparsity, sparsity_mbd=self.sparsity_mbd,sizes=sizes[2*i:(i+1)*2])
            for i in range(3)
        ])

#NOTE: con tutto true il fc ha bisogno di 1863, mentre nel caso false solo 1800
        self.fc = nn.Linear(1863, 1) 
        self.aux_fc = nn.Linear(1863, 1) if nb_classes > 1 else None

        # For minibatch discrimination
        self.dense3d_energy = Dense3D(first_dim=10, last_dim=10, input_shape=(3,1))

    def forward(self, inputs, input_energy):
        features = []
        energies = []

        # Extract features and energies from each calorimeter layer
        for i, input in enumerate(inputs):
            features.append(self.layers[i](input))
            energies.append(calculate_energy(input))

        # Concatenate features
        features = torch.cat(features, dim=1)
        energies = torch.cat(energies,dim=1)

        # Total energy across all rows
        total_energy = torch.sum(energies, dim=1, keepdim=True)

        # Minibatch discrimination on the raw energies
        K_energy = self.dense3d_energy(energies)
        mbd_energy = torch.tanh(minibatch_discriminator(K_energy))

        # Absolute deviation from input energy
        energy_well = torch.abs(total_energy - input_energy)
        
        # Binary y/n if it is over the input energy
        well_too_big = 10 * (energy_well > 5).float()

        # Concatenate all features
        p = torch.cat([
            features,
            scale(energies, 10),
            scale(total_energy, 100),
            energy_well,
            well_too_big,
            mbd_energy
        ], dim=1)

        fake = torch.sigmoid(self.fc(p))
        discriminator_outputs = [fake, total_energy]

        # Auxiliary output for ACGAN
        if self.nb_classes > 1:
            aux = torch.sigmoid(self.aux_fc(p))
            discriminator_outputs.append(aux)

        return discriminator_outputs

## Generatore

In [14]:
class Generator(nn.Module):
    def __init__(self, latent_size, no_attn, nb_classes=1):
        super(Generator, self).__init__()
        self.latent_size = latent_size
        self.nb_classes = nb_classes
        self.no_attn = no_attn

        # Embedding layer
        if nb_classes > 1:
            self.embedding = nn.Embedding(nb_classes, latent_size)
            self.flatten = nn.Flatten()


        # Define generator layers
        self.gen_layer0 = build_Generator(latent_size, 3, 96)
        self.gen_layer1 = build_Generator(latent_size, 12, 12)
        self.gen_layer2 = build_Generator(latent_size, 12, 6)

        if not no_attn:
            self.attn_layer1=InpaintingAttention(constant=-10.0, input_size=[14,14])
            self.attn_layer2=InpaintingAttention(constant=-10.0, input_size=[14,8])


    def forward(self, generator_inputs, image_class=None):
        latent=generator_inputs[0]
        input_energy=generator_inputs[1]
        if self.nb_classes > 1 and image_class is not None:
            emb = self.embedding(image_class)
            emb = self.flatten(emb)
            hc = latent * emb
            h = hc * scale(input_energy, 100)
        else:
            h = latent * scale(input_energy, 100).shape[1]

        img_layer0 = self.gen_layer0(h)
        img_layer1 = self.gen_layer1(h)
        img_layer2 = self.gen_layer2(h)

        if not no_attn:
            # resizes from (3, 96) => (12, 12)
            zero2one = nn.AvgPool2d(kernel_size=(1, 8))(
                nn.Upsample(scale_factor=(4, 1), mode='nearest')(img_layer0))
            img_layer1 = self.attn_layer1(img_layer1, zero2one)
            
            # resizes from (12, 12) => (12, 6)
            one2two = nn.AvgPool2d(kernel_size=(1, 2))(img_layer1)
            img_layer2 = self.attn_layer2(img_layer2, one2two)
     

        return [F.relu(img_layer0), F.relu(img_layer1), F.relu(img_layer2)]

## Combined model

In [15]:
inputs = [3,96,12,12,12,6]
discriminator = Discriminator(inputs,2)


for param in discriminator.parameters():
    param.requires_grad = False

latent = torch.randn(batch_size, latent_size)  # Example latent input
input_energy = torch.randn((batch_size, 1)) 
generator_inputs = [latent, input_energy] 
print("latent", latent.shape)
print("input",input_energy.shape)


generator=Generator(latent_size, False)
out=generator(generator_inputs)
#combined_input = [torch.cat([out[i], input_energy], dim=0) for i in range(3)]
#print(out[0].shape)
sampled_energies = torch.rand( size=(batch_size, 1))*99+1
out2=discriminator(out, sampled_energies)
print(out2[0].view(-1).shape)
print(out2[2].requires_grad)


class CombinedModel(nn.Module):
    def __init__(self, generator, discriminator):
        super(CombinedModel, self).__init__()
        self.generator = generator
        self.discriminator = discriminator

    def forward(self, generator_inputs, image_class=None):
        latent=generator_inputs[0]
        input_energy=generator_inputs[1]
        gen_output = self.generator(generator_inputs, image_class)
        disc_output = self.discriminator(gen_output, input_energy)
        return disc_output

# Instantiate the combined model


combined = CombinedModel(generator, discriminator)
out=combined(generator_inputs)
print("fake",out[0].shape)
print("energy", out[1].shape)
#summary(combined, input_size=[(1,latent_size), (1,1)])

latent torch.Size([256, 100])
input torch.Size([256, 1])
torch.Size([256])
True
fake torch.Size([256, 1])
energy torch.Size([256, 1])


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count parameters for each individual model
generator_params = count_parameters(generator)
discriminator_params = count_parameters(discriminator)
combined_params = count_parameters(combined)

print(f"Number of parameters in Generator: {generator_params}")
print(f"Number of parameters in Discriminator: {discriminator_params}")
print(f"Number of parameters in Combined Model: {combined_params}")

Number of parameters in Generator: 3177720
Number of parameters in Discriminator: 0
Number of parameters in Combined Model: 3177720


In [17]:
def calculate_memory_usage(model):
    total_memory = 0
    for param in model.parameters():
        if param.requires_grad:
            # The number of elements in the parameter
            num_elements = param.numel()
            # Size of each element in bytes (assuming float32, which is 4 bytes)
            element_size = param.element_size()
            # Total memory for this parameter
            total_memory += num_elements * element_size
    return total_memory

def format_memory_size(size_in_bytes):
    """ Convert the memory size from bytes to a readable format (KB, MB, GB) """
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_in_bytes < 1024:
            return f"{size_in_bytes:.2f} {unit}"
        size_in_bytes /= 1024

# Calculate memory usage for each individual model
generator_memory = calculate_memory_usage(generator)
discriminator_memory = calculate_memory_usage(discriminator)
combined_memory = calculate_memory_usage(combined)

print(f"Memory usage of Generator: {format_memory_size(generator_memory)}")
print(f"Memory usage of Discriminator: {format_memory_size(discriminator_memory)}")
print(f"Memory usage of Combined Model: {format_memory_size(combined_memory)}")


Memory usage of Generator: 12.12 MB
Memory usage of Discriminator: 0.00 B
Memory usage of Combined Model: 12.12 MB


## Train

In [18]:
class SaveBestModel:
    def __init__(self, best_valid_loss=float('inf')): #object initialized with best_loss = +infinite
        self.best_valid_loss = best_valid_loss

    def __call__(
        self, current_valid_loss,
        epoch, model, optimizer, criterion, metric
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss

            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")

            # method to save a model (the state_dict: a python dictionary object that
            # maps each layer to its parameter tensor) and other useful parametrers
            # see: https://pytorch.org/tutorials/beginner/saving_loading_models.html

            torch.save({'model' : model,
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,'metric': metric,
                }, 'best_model.pt')

In [19]:
opt_g = torch.optim.Adam(params=generator.parameters(), lr=gen_lr, weight_decay=1e-5)
opt_d = torch.optim.Adam(params=discriminator.parameters(), lr=disc_lr, weight_decay=1e-5)

bce_loss=nn.BCELoss()
mae_loss=nn.L1Loss()


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator.to(device)
discriminator.to(device)



model=CombinedModel(generator, discriminator)
model.to(device)

disc_loss = []
gen_loss = []

""" if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS device is available and being used")
else:
    device = torch.device("cpu")
    print("MPS device not available, using CPU")
 """

' if torch.backends.mps.is_available():\n    device = torch.device("mps")\n    print("MPS device is available and being used")\nelse:\n    device = torch.device("cpu")\n    print("MPS device not available, using CPU")\n '

In [20]:
nb_epochs=1
for epoch in range(nb_epochs):
    t0 = time.time()
    train_loss = 0
    counter=0
    disc_loss_partial=0
    gen_loss_partial=0
    for image_batch_1,image_batch_2, image_batch_3, energy_batch, label_batch in tqdm(dataloader, desc="Training"):
        time.sleep(0.1)

        noise = torch.normal(0, 1, size=(batch_size, latent_size), requires_grad=True)

        # energy_breakdown

        sampled_labels = torch.randint(0, nb_classes, size=(batch_size,))
        sampled_energies = torch.rand( size=(batch_size, 1))*99+1

        generator_inputs = [noise, sampled_energies]

        """ if nb_classes > 1:
                # in the case of the ACGAN, we need to append the requested
                # class to the pre-image of the generator
                generator_inputs.append(sampled_labels) """

        generated_images = generator(generator_inputs)

        #disc_outputs_real = [torch.ones(batch_size), energy_batch]
        #disc_outputs_fake = [torch.zeros(batch_size), sampled_energies]

        #loss_weights = torch.Tensor([1, 0.05]
        #print(loss_weights.requires_grad)

        
        
        out = discriminator([image_batch_1,image_batch_2, image_batch_3], energy_batch)
        loss_real =   bce_loss(out[0].view(-1), torch.ones(batch_size)) + 0.05 * mae_loss(out[1].view(-1), energy_batch)
        
        opt_d.zero_grad()
        loss_real.backward()
        opt_d.step()

        out = discriminator(generated_images, sampled_energies)
        loss_fake =  bce_loss(out[0].view(-1), torch.zeros(batch_size)) + 0.05*mae_loss(out[1].view(-1), sampled_energies)
        
        opt_d.zero_grad()
        loss_fake.backward()
        opt_d.step()

        disc_loss_partial+=((loss_real.item() + loss_fake.item()) / 2)
        

        trick = torch.ones(batch_size)

        for _ in range(2):
            noise = torch.normal(0, 1, size=(batch_size, latent_size), requires_grad=True)

            sampled_energies = torch.rand( size=(batch_size, 1))*99+1
            combined_inputs = [noise, sampled_energies]
            combined_outputs = [trick, sampled_energies]
            if nb_classes > 1:
                sampled_labels = np.random.randint(0, nb_classes,
                                                   batch_size)
                combined_inputs.append(sampled_labels)
                combined_outputs.append(sampled_labels)

            out=model(combined_inputs)
            gen_loss_partial+=bce_loss(out[0].view(-1), trick) + 0.05*mae_loss(out[1].view(-1), sampled_energies)

disc_loss.append(disc_loss_partial/batch_size)
gen_loss.append(gen_loss_partial/batch_size)
print('Epoch {:3d} Generator loss: {}'.format(epoch + 1, gen_loss_partial/batch_size))
print(('Epoch {:3d} Discriminator loss: {}'.format(epoch + 1, disc_loss_partial/batch_size)))
        
       


        


        


Training:   0%|          | 0/39 [00:00<?, ?it/s]/Users/martinachirico/miniconda3/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
Training:   5%|▌         | 2/39 [00:09<03:00,  4.87s/it]


KeyboardInterrupt: 